In [ ]:
!pip install torch

In [ ]:
!pip install transformers
!pip install tokenizers
!pip install sentencepiece

In [ ]:
# Importing stock libraries
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from string import punctuation
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

import textwrap

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer
import json
import pandas as pd

# Load pre-trained T5 model and tokenizer
model_name = "T5_BIO"  # Use the name you saved the model with
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

# Load your training data
file_path = 'Users/21001279007/training10b.json'  # Replace with the actual path to your JSON file
with open(file_path, 'r') as json_file:
    json_data = json.load(json_file)

questions_list = []
for question in json_data['questions']:
    question_text = question['body']
    ideal_answer = question['ideal_answer'][0]

    context = ""
    for snippet in question['snippets']:
        snippet_text = snippet['text']
        context += snippet_text + " "

    questions_list.append({
        'Ideal_Answer': ideal_answer,
        'Combined_Text': question_text + ' ' + context.strip()
    })

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(questions_list)

def convert_examples_to_features(batch):
    input_encodings = tokenizer(batch['Combined_Text'], max_length=1024, truncation=True)
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(batch['Ideal_Answer'], max_length=1024, truncation=True)

    return {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

# Apply the convert_examples_to_features function to each row
dataset_pt = df.apply(convert_examples_to_features, axis=1)

# Convert the result into a list of dictionaries
dataset_pt = dataset_pt.to_list()

# Define the data collator for seq2seq
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Define training arguments for additional 2 epochs
trainer_args = TrainingArguments(
    output_dir='BIO_T5_Large2',  # Use the same output directory
    num_train_epochs=1,  # Train for 2 more epochs
    warmup_steps=500,
    per_device_train_batch_size=1,
    weight_decay=0.01,
    logging_steps=10,
    evaluation_strategy='steps',
    eval_steps=500,
    save_steps=1e6,
    gradient_accumulation_steps=16
)

# Create the Trainer instance with your model and new training arguments
trainer = Trainer(
    model=model,
    args=trainer_args,
    tokenizer=tokenizer,
    train_dataset=dataset_pt,
    data_collator=seq2seq_data_collator,
)

# Fine-tune for 2 more epochs
trainer.train()

# Save the updated model
trainer.save_model('T5_BIO2')


2023-12-20 08:40:28.262373: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-20 08:40:30.723954: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-20 08:40:31.409204: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-12-20 08:40:31.409227: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

Step,Training Loss,Validation Loss


Attempted to log scalar metric loss:
0.8663
Attempted to log scalar metric learning_rate:
1.0000000000000002e-06
Attempted to log scalar metric epoch:
0.04
Attempted to log scalar metric loss:
0.7628
Attempted to log scalar metric learning_rate:
2.0000000000000003e-06
Attempted to log scalar metric epoch:
0.08
Attempted to log scalar metric loss:
0.7198
Attempted to log scalar metric learning_rate:
3e-06
Attempted to log scalar metric epoch:
0.11
Attempted to log scalar metric loss:
0.7139
Attempted to log scalar metric learning_rate:
4.000000000000001e-06
Attempted to log scalar metric epoch:
0.15
Attempted to log scalar metric loss:
0.7648
Attempted to log scalar metric learning_rate:
5e-06
Attempted to log scalar metric epoch:
0.19
Attempted to log scalar metric loss:
0.7066
Attempted to log scalar metric learning_rate:
6e-06
Attempted to log scalar metric epoch:
0.23
Attempted to log scalar metric loss:
0.7338
Attempted to log scalar metric learning_rate:
7.000000000000001e-06
Atte



Training completed. Do not forget to share your model on huggingface.co/models =)




Attempted to log scalar metric train_runtime:
36390.6959
Attempted to log scalar metric train_samples_per_second:
0.116
Attempted to log scalar metric train_steps_per_second:
0.007
Attempted to log scalar metric total_flos:
1.0309834746630144e+16
Attempted to log scalar metric train_loss:
0.6508506301677588
Attempted to log scalar metric epoch:
1.0


Saving model checkpoint to T5_BIO2
Configuration saved in T5_BIO2/config.json
Model weights saved in T5_BIO2/pytorch_model.bin
tokenizer config file saved in T5_BIO2/tokenizer_config.json
Special tokens file saved in T5_BIO2/special_tokens_map.json


In [ ]:
from transformers import pipeline
import json
import pandas as pd
import os

# Create a summarization pipeline
pipe = pipeline('summarization', model='T5_BIO2')

gen_kwargs = {'length_penalty': 0.10, 'num_beams': 8, 'max_length': 1024, 'truncation': True}

# Specify the output folder
output_folder = 'system6(2)'
os.makedirs(output_folder, exist_ok=True)

# Open and read the JSON file
file_path = '10B6_golden.json'  # Replace with the actual path to your JSON file
with open(file_path, 'r') as json_file:
    json_data = json.load(json_file)

questions_list = []

# Extract data from the JSON and create a list of dictionaries
for index, question in enumerate(json_data['questions'], start=1):
    question_text = question['body']
    ideal_answer = question['ideal_answer'][0]

    context = ""
    for snippet in question['snippets']:
        snippet_text = snippet['text']
        context += snippet_text + " "

    questions_list.append({
        'Question_Number': index,
        'Ideal_Answer': ideal_answer,
        'Combined_Text': question_text + ' ' + context.strip()
    })

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(questions_list)

# Loop through the DataFrame
for index, row in df.iterrows():
    question_number = row['Question_Number']
    question_text = row['Combined_Text']

    generated_summary = pipe(question_text, **gen_kwargs)[0]['summary_text']

    # Save the generated summary to a separate file
    output_file_path = f"{output_folder}/task{question_number}_englishSyssum{question_number}.txt"
    with open(output_file_path, 'w') as output_file:
        output_file.write(generated_summary)

    print(f"Generated summary for Question Number {question_number} saved to {output_file_path}")


loading configuration file T5_BIO2/config.json
Model config T5Config {
  "_name_or_path": "T5_BIO2",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2816,
  "d_kv": 64,
  "d_model": 1024,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 24,
  "num_heads": 16,
  "num_layers": 24,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.16.0",
  "use_cache": true,
  "vocab_size": 32128
}

loading configuration file T5_BIO2/config.json
Model config T5Config {
  "_name_or_path": "T5_BIO2",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2816,
  "d_kv": 64,
  "d_model": 1024,
 

Generated summary for Question Number 1 saved to system6(2)/task1_englishSyssum1.txt


Your max_length is set to 1024, but you input_length is only 158. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=79)


Generated summary for Question Number 2 saved to system6(2)/task2_englishSyssum2.txt


Your max_length is set to 1024, but you input_length is only 345. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=172)


...

Generated summary for Question Number 34 saved to system6(2)/task34_englishSyssum34.txt


Your max_length is set to 1024, but you input_length is only 512. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=256)


Generated summary for Question Number 35 saved to system6(2)/task35_englishSyssum35.txt


Your max_length is set to 1024, but you input_length is only 512. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=256)


Generated summary for Question Number 36 saved to system6(2)/task36_englishSyssum36.txt
Generated summary for Question Number 37 saved to system6(2)/task37_englishSyssum37.txt


In [ ]:
import shutil

# Zip the 'system' folder
shutil.make_archive('system6_2', 'zip', 'system6(2)')

print("Folder 'system6(2)' has been zipped.")

Folder 'system6(2)' has been zipped.


In [ ]:
from transformers import pipeline
import json
import pandas as pd
import os

# Create a summarization pipeline
pipe = pipeline('summarization', model='T5_BIO2')

gen_kwargs = {'length_penalty': 0.10, 'num_beams': 8, 'max_length': 1024, 'truncation': True}

# Specify the output folder
output_folder = 'system5(2)'
os.makedirs(output_folder, exist_ok=True)

# Open and read the JSON file
file_path = '10B5_golden.json'  # Replace with the actual path to your JSON file
with open(file_path, 'r') as json_file:
    json_data = json.load(json_file)

questions_list = []

# Extract data from the JSON and create a list of dictionaries
for index, question in enumerate(json_data['questions'], start=1):
    question_text = question['body']
    ideal_answer = question['ideal_answer'][0]

    context = ""
    for snippet in question['snippets']:
        snippet_text = snippet['text']
        context += snippet_text + " "

    questions_list.append({
        'Question_Number': index,
        'Ideal_Answer': ideal_answer,
        'Combined_Text': question_text + ' ' + context.strip()
    })

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(questions_list)

# Loop through the DataFrame
for index, row in df.iterrows():
    question_number = row['Question_Number']
    question_text = row['Combined_Text']

    generated_summary = pipe(question_text, **gen_kwargs)[0]['summary_text']

    # Save the generated summary to a separate file
    output_file_path = f"{output_folder}/task{question_number}_englishSyssum{question_number}.txt"
    with open(output_file_path, 'w') as output_file:
        output_file.write(generated_summary)

    print(f"Generated summary for Question Number {question_number} saved to {output_file_path}")


loading configuration file T5_BIO2/config.json
Model config T5Config {
  "_name_or_path": "T5_BIO2",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2816,
  "d_kv": 64,
  "d_model": 1024,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 24,
  "num_heads": 16,
  "num_layers": 24,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.16.0",
  "use_cache": true,
  "vocab_size": 32128
}

loading configuration file T5_BIO2/config.json
Model config T5Config {
  "_name_or_path": "T5_BIO2",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2816,
  "d_kv": 64,
  "d_model": 1024,
 

Generated summary for Question Number 1 saved to system5(2)/task1_englishSyssum1.txt


Your max_length is set to 1024, but you input_length is only 140. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=70)


Generated summary for Question Number 2 saved to system5(2)/task2_englishSyssum2.txt


Your max_length is set to 1024, but you input_length is only 512. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=256)


...

Generated summary for Question Number 87 saved to system5(2)/task87_englishSyssum87.txt


Your max_length is set to 1024, but you input_length is only 177. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=88)


Generated summary for Question Number 88 saved to system5(2)/task88_englishSyssum88.txt


Your max_length is set to 1024, but you input_length is only 512. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=256)


Generated summary for Question Number 89 saved to system5(2)/task89_englishSyssum89.txt
Generated summary for Question Number 90 saved to system5(2)/task90_englishSyssum90.txt


In [ ]:
import shutil

# Zip the 'system' folder
shutil.make_archive('system5_2', 'zip', 'system5(2)')

print("Folder 'system5(2)' has been zipped.")

Folder 'system5(2)' has been zipped.
